In [19]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Preprocess the dataset

In [20]:
# load dataset
imsize = 224
image_dir = '../train/'
metadata = pd.read_csv('../train.csv')
metadata['filename'] = metadata['id'].apply(lambda x: f"{image_dir}{x}.jpg")
train_df, val_df = train_test_split(metadata, test_size=0.2, random_state=42)

In [21]:
metadata['stable_height'].unique() # six label in total

array([2, 1, 5, 6, 4, 3])

In [22]:
import random
random_seed = 25
random.seed(random_seed)
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

In [23]:
# augmentataion, only apply to the training set
def preprocess(filepath, label):
    image = tf.io.read_file(filepath) # load file
    image = tf.image.decode_jpeg(image, channels=3) # decode it into a tensor
    image = tf.image.resize(image, [224, 224])
    image = (image - 127.5) / 127.5
    return image, label - 1

In [24]:
# Apply augementation on training set and normalize the validation set
train_ds = tf.data.Dataset.from_tensor_slices((train_df['filename'], train_df['stable_height']))
train_ds = train_ds.map(preprocess).batch(32).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_df['filename'], val_df['stable_height']))
val_ds = val_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [25]:
for images, labels in train_ds.take(1):  # Just take one batch, check the info 
    print("Image batch shape: ", images.shape)
    print("Label batch shape: ", labels.shape)
    print("Labels: ", labels)

Image batch shape:  (15, 224, 224, 3)
Label batch shape:  (15,)
Labels:  tf.Tensor([4 2 4 2 2 0 0 3 2 0 2 0 3 1 5], shape=(15,), dtype=int64)


2024-10-18 16:18:37.037382: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# train

In [11]:
from tensorflow.keras.applications import ResNet152, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras

# Load ResNet-152 or ResNet-50 here pre-trained on ImageNet without the top layer
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(imsize, imsize, 3))

for layer in base_model.layers:
    layer.trainable = True 

# Add new layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(6, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model with both callbacks
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/100


I0000 00:00:1729206610.064372   75967 service.cc:146] XLA service 0x7f3ff0002d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729206610.064427   75967 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-10-18 10:10:11.346703: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-18 10:10:20.140857: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45017', 196 bytes spill stores, 196 bytes spill loads

2024-10-18 10:10:42.851021: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_76', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1729206643.188393   75

192/192 ━━━━━━━━━━━━━━━━━━━━ 120s 230ms/step - accuracy: 0.2348 - loss: 1.8623 - val_accuracy: 0.2415 - val_loss: 3.8356 - learning_rate: 5.0000e-04
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/step - accuracy: 0.3409 - loss: 1.4816 - val_accuracy: 0.2415 - val_loss: 1.9192 - learning_rate: 5.0000e-04
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 39s 204ms/step - accuracy: 0.4356 - loss: 1.3290 - val_accuracy: 0.2396 - val_loss: 1.9339 - learning_rate: 5.0000e-04
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/step - accuracy: 0.5142 - loss: 1.1795 - val_accuracy: 0.3691 - val_loss: 1.5799 - learning_rate: 5.0000e-04
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/step - accuracy: 0.5657 - loss: 1.0618 - val_accuracy: 0.4772 - val_loss: 1.3806 - learning_rate: 5.0000e-04
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/step - accuracy: 0.6228 - loss: 0.9674 - val_accuracy: 0.4134 - val_loss: 1.4382 - learning_rate: 5.0000e-04
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/

In [12]:
model.save("../model/resnet152.keras")